# Config

In [1]:
import torch.nn as nn

NB_SAMPLES = 1000
DATA_DIR = './data'

NUMBER_OF_CLASSES = 10

WIDTH_HEIGHT = 14
SINGLE_IMAGE_SIZE = WIDTH_HEIGHT * WIDTH_HEIGHT
DOUBLE_IMAGE_SIZE = 2 * SINGLE_IMAGE_SIZE

# ----Train Config-----#
LEARNING_RATE = 0.001
TRAIN_BATCH_SIZE = 1
SUB_CRITERION = nn.CrossEntropyLoss()
FINAL_CRITERION = nn.BCELoss()
EPOCHS = 20

TRAIN_CHECKPOINTS_DIR = "./checkpoints"
SAVE_MODEL_EVERY_X_EPOCH = 5

# ----AuxLoss Config-----#
ALPHA = 0.5
BETA = 0.25
GAMMA = 0.25
WEIGHTS_LOSS = ALPHA, BETA, GAMMA

#----Test Config-----#
TEST_BATCH_SIZE = NB_SAMPLES

#----BasicNet Config-----#
BASIC_NET_NAME = "basic_net"
BASIC_NET_HIDDEN_LAYER = 512

#----OscarNet Config-----#
OSCAR_NET_NAME = "oscar_net"
OSCAR_NET_HIDDEN_LAYER = 512

#----DesmondNet Config-----#
DESMOND_NET_NAME = "desmond_net"
DESMOND_NET_HIDDEN_LAYER = 512

#----RobertNet Config-----#
ROBERT_NET_NAME = "robert_net"
ROBERT_NET_HIDDEN_LAYER = 512
ROBERT_NET_BASE_CHANNEL_SIZE = 8

#----LeonardtNet Config-----#
LEONARD_NET_NAME = "leonard_net"
LEONARD_NET_HIDDEN_LAYER = 512

# Helpers

In [2]:
############################################# HELPERS ###############################

import torch
from torchvision import datasets
import os
import time
from pathlib import Path


######################################################################
# The data

def convert_to_one_hot_labels(input, target):
    tmp = input.new_zeros(target.size(0), target.max() + 1)
    #set ones
    tmp.scatter_(1, target.view(-1, 1), 1.0)
    return tmp

def load_data(cifar = None, one_hot_labels = False, normalize = False, flatten = True):

    data_dir = './data'

    if (cifar is not None and cifar):
        print('* Using CIFAR')
        cifar_train_set = datasets.CIFAR10(data_dir + '/cifar10/', train = True, download = True)
        cifar_test_set = datasets.CIFAR10(data_dir + '/cifar10/', train = False, download = True)

        train_input = torch.from_numpy(cifar_train_set.data)
        train_input = train_input.transpose(3, 1).transpose(2, 3).float()
        train_target = torch.tensor(cifar_train_set.targets, dtype = torch.int64)

        test_input = torch.from_numpy(cifar_test_set.data).float()
        test_input = test_input.transpose(3, 1).transpose(2, 3).float()
        test_target = torch.tensor(cifar_test_set.targets, dtype = torch.int64)

    else:
        print('* Using MNIST')
        mnist_train_set = datasets.MNIST(data_dir + '/mnist/', train = True, download = True)
        mnist_test_set = datasets.MNIST(data_dir + '/mnist/', train = False, download = True)

        train_input = mnist_train_set.data.view(-1, 1, 28, 28).float()
        train_target = mnist_train_set.targets
        test_input = mnist_test_set.data.view(-1, 1, 28, 28).float()
        test_target = mnist_test_set.targets

    if flatten:
        train_input = train_input.clone().reshape(train_input.size(0), -1)
        test_input = test_input.clone().reshape(test_input.size(0), -1)
        
        
    train_input = train_input.narrow(0, 0, 1000)
    train_target = train_target.narrow(0, 0, 1000)
    test_input = test_input.narrow(0, 0, 1000)
    test_target = test_target.narrow(0, 0, 1000)

    print('** Use {:d} train and {:d} test samples'.format(train_input.size(0), test_input.size(0)))

    if one_hot_labels:
        train_target = convert_to_one_hot_labels(train_input, train_target)
        test_target = convert_to_one_hot_labels(test_input, test_target)

    if normalize:
        mu, std = train_input.mean(), train_input.std()
        train_input.sub_(mu).div_(std)
        test_input.sub_(mu).div_(std)

    return train_input, train_target, test_input, test_target

######################################################################

def mnist_to_pairs(nb, input, target):
    input = torch.functional.F.avg_pool2d(input, kernel_size = 2)
    a = torch.randperm(input.size(0))
    a = a[:2 * nb].view(nb, 2)
    input = torch.cat((input[a[:, 0]], input[a[:, 1]]), 1)
    classes = target[a]
    target = (classes[:, 0] <= classes[:, 1]).long()
    return input, target, classes

######################################################################

def generate_pair_sets(nb):

    data_dir = DATA_DIR

    train_set = datasets.MNIST(data_dir + '/mnist/', train = True, download = True)
    train_input = train_set.data.view(-1, 1, 28, 28).float()
    train_target = train_set.targets

    test_set = datasets.MNIST(data_dir + '/mnist/', train = False, download = True)
    test_input = test_set.data.view(-1, 1, 28, 28).float()
    test_target = test_set.targets

    return mnist_to_pairs(nb, train_input, train_target) + \
           mnist_to_pairs(nb, test_input, test_target)

######################################################################

def compute_accuracy(tensor1, tensor2):
    
    tensor_accuracy = torch.where(tensor1 == tensor2, torch.tensor(1), torch.tensor(0))
    
    accuracy = torch.sum(tensor_accuracy).item() / NB_SAMPLES
    
    return accuracy

######################################################################

def save_model(model, epoch=None, loss=None, save_dir=None, specific_name=None):

    if epoch and loss and save_dir and specific_name:
        model_name = model.model_name
        timestr = time.strftime("%Y%m%d-%H%M%S")
        file_name = f"{timestr}_{model_name}_epoch_{epoch}_loss_{loss:03.3f}.pt"
        Path(save_dir).mkdir(exist_ok=True)
        file_path = Path(save_dir) / file_name
        torch.save(model.state_dict(), str(file_path))
    elif save_dir and specific_name:
        file_path = Path(save_dir) / specific_name
        torch.save(model.state_dict(), str(file_path))

# Siamese

In [3]:
###############################3 SIAMESE ###########################################3
import torch.nn as nn
import torch
from torch.nn import functional as F
import math

class siamese_net(nn.Module):

    def __init__(self, weight_sharing = True, architecture = 1, nb_channels = 24, nb_hidden = 300, k = 1):
        
        super(siamese_net, self).__init__()
        
        #if weight sharing forward on the same subnet
        if weight_sharing: 
            self.nb_subnet = 1
        else:
            self.nb_subnet = 2
        
        if (architecture == 1 ):
            # subnetwork 1
            # (1,14,14) conv (1,3,3) =>  (1,12,12)
            # Relu 
            # (1,12,12) conv (1,3,3) => (1,10,10)
            # Relu
            # (1,10,10) maxpool (2,2) => (5,5)
            # try to add different channels and add dropout ? 
            self.conv = nn.ModuleList([nn.Sequential(nn.Conv2d(1, nb_channels , kernel_size=3),  
                                   nn.LeakyReLU(),
                                   nn.Dropout(p=0.2),                  
                                   nn.Conv2d(nb_channels, 2 * nb_channels, kernel_size=3),
                                   nn.LeakyReLU(),           
                                   nn.MaxPool2d(2),
                                   nn.Dropout(p=0.2)) for i in range(self.nb_subnet) ] )
        
            #fully connected part   
            # from a conv (5,5)  => fc layer (25,300) => Relu => fc layer (300,10) 
            
            out_conv = 25 * 2 * nb_channels                                          
            
            self.fc = nn.ModuleList([nn.Sequential(nn.Linear(out_conv , nb_hidden), nn.LeakyReLU(), nn.Dropout(p=0.2),
                                                   nn.Linear(nb_hidden, 10) ) for i in range(self.nb_subnet) ] )
        if (architecture == 2 ):
            #(W−F+2P)/S+1
            
            self.conv = nn.ModuleList([nn.Sequential(nn.Conv2d(1, nb_channels, kernel_size=5, stride=1, padding=1), #(14 - 5 + 2 ) + 1 =12
                                                     nn.LeakyReLU(),
                                                     nn.MaxPool2d(kernel_size=3, stride=3), #4
                                                     nn.Dropout(p=0.2),
                                                     nn.Conv2d(nb_channels, 2 * nb_channels, 
                                                               kernel_size=3, stride=1, padding=1), #(4-3 +2) + 1 = 4
                                                     nn.LeakyReLU(),
                                                     nn.Dropout(p=0.2)) for i in range(self.nb_subnet) ] )
            
            out_conv = 4 * 4 * 2 * nb_channels #flatten the (2,2, 2 * nb_channels) tensor                                         
            
            self.fc = nn.ModuleList([nn.Sequential(nn.Linear(out_conv , nb_hidden), nn.LeakyReLU(), nn.Dropout(p=0.2),
                                                   nn.Linear(nb_hidden, 10) ) for i in range(self.nb_subnet) ] )

        if (architecture == 3 ):
            
            #(W−F+2P)/S+1
           
            self.conv = nn.ModuleList([nn.Sequential(nn.Conv2d(1, nb_channels, kernel_size=3, stride=1, padding=0),  #(14-3+0) + 1 = 12 
                                                     nn.LeakyReLU(),
                                                     nn.MaxPool2d(kernel_size=3, stride=3), #4
                                                     nn.Dropout(p=0.2),
                                                     nn.Conv2d(nb_channels, 2 * nb_channels, kernel_size=2, stride=1, padding=0), #3
                                                     nn.LeakyReLU(),
                                                     nn.Dropout(p=0.2)) for i in range(self.nb_subnet) ] )
                                                     
            out_conv = 3* 3 * 2 * nb_channels                                          
            
            self.fc = nn.ModuleList([nn.Sequential(nn.Linear(out_conv , nb_hidden), nn.LeakyReLU(), nn.Dropout(p=0.2),
                                                   nn.Linear(nb_hidden, 10) ) for i in range(self.nb_subnet) ] )
        
        if (architecture == 4 ):
            
            #(W−F+2P)/S+1
            self.conv = nn.ModuleList([nn.Sequential(nn.Conv2d(1, nb_channels, kernel_size=2, stride=1, padding=1),  #(14-2+2)+1 = 15
                                                     nn.LeakyReLU(),
                                                     nn.MaxPool2d(kernel_size=3, stride=3), #5
                                                     nn.Dropout(p=0.2),
                                                     nn.Conv2d(nb_channels, 2 * nb_channels, 
                                                               kernel_size=3, stride=1, padding=0), #(5-3)+1 = 3
                                                     nn.LeakyReLU(),
                                                     #nn.MaxPool2d(kernel_size=3, stride=3), #1  
                                                     nn.Dropout(p=0.2) ) for i in range(self.nb_subnet) ] )
            
            out_conv = 3* 3 * 2 * nb_channels 
            
            self.fc = nn.ModuleList([nn.Sequential(nn.Linear(out_conv , nb_hidden), nn.LeakyReLU(), nn.Dropout(p=0.2),
                                                   nn.Linear(nb_hidden, 10) ) for i in range(self.nb_subnet) ] )
        if (architecture == 5 ):
            
            #(W−F+2P)/S+1
            #k= 1, 3, 5  
            self.conv = nn.ModuleList([nn.Sequential(nn.Conv2d(1, nb_channels, kernel_size= k ),  #(14-k)+1 = 15 -k # 14, 12,10
                                                     nn.LeakyReLU(),
                                                     nn.MaxPool2d(kernel_size=2, stride=2), #(15-k) / 2 : 7, 6, 5  
                                                     nn.Dropout(p=0.2),
                                                     nn.Conv2d(nb_channels, 2 * nb_channels, 
                                                               kernel_size = k),    #7, 5, 3   # (15 - k) /2 - k + 1
                                                     nn.LeakyReLU(),
                                                     #nn.MaxPool2d(kernel_size=3, stride=3), #1  
                                                     nn.Dropout(p=0.2) ) for i in range(self.nb_subnet) ] )
           
            out_conv = int( ( ((15 - k) /2 - k + 1)** 2 ) * 2 * nb_channels)
      
            self.fc = nn.ModuleList([nn.Sequential(nn.Linear(out_conv , nb_hidden), nn.LeakyReLU(), nn.Dropout(p=0.2),
                                                   nn.Linear(nb_hidden, 10) ) for i in range(self.nb_subnet) ] )
                
        #combine the subnetwork output
        #(20,1) and sigmoid
        self.comb = nn.Sequential(nn.Linear(20, 256), nn.LeakyReLU(), nn.Dropout(p=0.2),
                                  nn.Linear(256, 256), nn.LeakyReLU(), nn.Dropout(p=0.2),
                                  nn.Linear(256, 1), nn.Sigmoid() ) 
     
            
    def forward_once(self, x, subnet_nbr):
        
        x = self.conv[subnet_nbr](x)  #(batch_size,1,5,5)
        
        #need to flatten to insert in the fc layer
        x = x.view(x.size(0), -1)    #(batch_size,25)
        
        x = self.fc[subnet_nbr](x)   #(batch_size,10)
        
        return x
    
    def forward(self, input1, input2): 
        
        #we compute the output for each subnet 
        #apply conv and fc and get 10 units per subnetwork
        output1 = self.forward_once(input1,0)
        output2 = self.forward_once(input2, self.nb_subnet - 1)  #-1 because we access a list index starting at 0 
        
        #concat the two output 
        concat_subnet = torch.cat((output1, output2),1)     #get (batch_size,20)
        
        #forward on combination layer
        output = self.comb(concat_subnet)
        
        return output1, output2, output

# Train

In [4]:
import torch.nn as nn
import torch
from torch.nn import functional as F
import math
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR


def one_hot_encoding(input_, nb_classes): 
    tmp = input_.new_zeros(input_.size()[0], nb_classes)  #tensor of zeros (batch_size,10)
    tmp.scatter_(1, input_, 1)     #fill with one in the correct index
    return tmp

def train_siamese(model, dataloader, test_data, epochs, learning_rate, aux_loss = False, alpha = 0.5 ):
    
    cuda = torch.cuda.is_available()
    if cuda:
        model = model.to(device="cuda")
#         print("CUDA available")
#     else:
#         print("NO CUDA")
        
        
    model.train() #set the model on training mode 
    
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr = learning_rate)
    
    # gamma is the decaying factor, after every 1 epoch new_lr = lr*gamma 
    scheduler = StepLR(optimizer, step_size=1, gamma = 0.9)
    
    if(aux_loss):
        criterion_aux = nn.CrossEntropyLoss()
        
    print("Start training with {0} epochs, a learning rate of {1} and {2} as loss function".format(epochs,learning_rate, criterion))
        
    if(aux_loss):
        print("With auxilary loss function")
    else: 
        print("Without auxilary loss function")
            
    if(model.nb_subnet == 1):
        print("With weight sharing")
    else:
        print("Without weight sharing")
           
    training_losses = []
    training_acc = []
    test_losses = []
    test_acc = []
    
    for epoch in range(1, epochs+1):
    
        sum_loss_epoch = 0
        total = 0
        correct = 0
        accuracy_epoch = 0
        
        for ind_batch, sample_batched in enumerate(dataloader):
    
            images = sample_batched["images"]                                  #(batch_size,2,14,14)
            compare_labels = sample_batched["bool_labels"].float().view(-1,1)  #(batch_size,1)
            digit_labels = sample_batched["digit_labels"]                      #(batch_size,2)
            
            if cuda:
                images = images.to(device="cuda")
                compare_labels = compare_labels.to(device="cuda")
                digit_labels = digit_labels.to(device = "cuda")
                
            #gets (batch_size,1) and returns (batch_size,10)
            one_hot_encoded_label1 = one_hot_encoding(digit_labels[:,0].view(-1,1), nb_classes=10)
            one_hot_encoded_label2 = one_hot_encoding(digit_labels[:,1].view(-1,1), nb_classes=10)
            
            loss = 0
            optimizer.zero_grad()
        
            #seperate the 2 batches of images
            input1 = images[:,0:1,:,:]   #(batch_size,1,14,14)
            input2 = images[:,1:2,:,:]   #(batch_size,1,14,14)
        
            #compute the forward pass
            output1, output2, output = model.forward(input1,input2)
            
            if(aux_loss):   
                #Cross entropy loss but as already applying the soft_max activation function
                #so only need to apply log
                #add small value to avoid the log(0) problem
                #multiplication element wise to keep only the probability of the correct label
            
                #loss_input1 = -torch.log(output1 + 1e-20) * one_hot_encoded_label1.float()   #(batch_size,1)
                #loss_input2 = -torch.log(output2 + 1e-20) * one_hot_encoded_label2.float()   #(batch_size,1)
                #loss = weight_loss_1 * loss_input1.mean() + weight_loss_2 * loss_input2.mean()  #auxilary loss
                
                loss1 =  criterion_aux(output1,digit_labels[:,0])
                loss2 = criterion_aux(output2,digit_labels[:,1])
                loss = (1 - alpha)/2 * loss1 + (1 - alpha)/2 * loss2 

            loss3 = criterion(output, compare_labels)   #if batched do the mean of the errors 
            
            loss += alpha * loss3

            loss.require_grad = True   #should remove I think
            loss.backward()

            optimizer.step()
            
            #update the accuracy 
            total += images.size(0)  
            correct += (output.round() == compare_labels).sum()  
            
            #if ind_batch % 1000 == 0:
            #    print("[Epoch {}, Batch {}/{}]:  [Loss: {:.2f}]".format(epoch, ind_batch, len(dataloader), loss) )
                
            #add the loss for this batch to the total loss of the epoch
            sum_loss_epoch = sum_loss_epoch + loss.item()
        
        scheduler.step()
        
        #compute the mean to obtain the loss for this epoch 
        mean_loss = sum_loss_epoch / float(len(dataloader))
        
        print('epoch {0}/{1}'.format(epoch, epochs))
        
#         print("At epoch {0} the loss is {1}".format(epoch, mean_loss) )
        training_losses.append(mean_loss)
        
        accuracy_epoch = float(correct) / float(total)
#         print("At epoch {0} the accuracy is {1}".format(epoch, accuracy_epoch) )
        training_acc.append(accuracy_epoch)
        
        test_loss, test_accuracy = test_siamese(model, test_data, aux_loss, alpha)

        test_losses.append(test_loss)
        test_acc.append(test_accuracy)
        
    return training_losses, training_acc, test_losses, test_acc

def test_siamese(model, dataloader, aux_loss = False, alpha = 0.5):
    
    cuda = torch.cuda.is_available()
    if cuda:
        model = model.to(device="cuda")
#         print("CUDA available")
#     else:
#         print("NO CUDA")
        
    model.eval() # set the model on evaluation mode
    
    criterion = nn.BCELoss()
    
    sum_test_loss = 0
    total = 0
    correct = 0
    
    if(aux_loss):
        criterion_aux = nn.CrossEntropyLoss()
        
    for ind_batch, sample_batched in enumerate(dataloader):
        
        images = sample_batched["images"]                                  #(batch_size,2,14,14)
        compare_labels = sample_batched["bool_labels"].float().view(-1,1)  #(batch_size,1)
        digit_labels = sample_batched["digit_labels"]                      #(batch_size,2)    
        
        if cuda:
                images = images.to(device="cuda")
                compare_labels = compare_labels.to(device="cuda")
                digit_labels = digit_labels.to(device = "cuda")
                
        #gets (batch_size,1) and returns (batch_size,10)
        one_hot_encoded_label1 = one_hot_encoding(digit_labels[:,0].view(-1,1), nb_classes=10)
        one_hot_encoded_label2 = one_hot_encoding(digit_labels[:,1].view(-1,1), nb_classes=10)        
        
        #seperate the 2 batches of images
        input1 = images[:,0:1,:,:]   #(batch_size,1,14,14)
        input2 = images[:,1:2,:,:]   #(batch_size,1,14,14)
        
        #compute the forward pass
        output1, output2, output = model.forward(input1,input2)
        
        if(aux_loss):
            #loss_input1 = -torch.log(output1 + 1e-20) * one_hot_encoded_label1.float()   #(batch_size,1)
            #loss_input2 = -torch.log(output2 + 1e-20) * one_hot_encoded_label2.float()   #(batch_size,1)
            #sum_test_loss += weight_loss_1 * loss_input1.mean() + weight_loss_2 * loss_input2.mean()  #auxilary loss
            loss1 =  criterion_aux(output1,digit_labels[:,0])
            loss2 = criterion_aux(output2,digit_labels[:,1])
            sum_test_loss = (1 - alpha)/2 * loss1 + (1 - alpha)/2 * loss2  
  
        sum_test_loss += criterion(output, compare_labels) 
         
        total += images.size(0)  
        correct += (output.round() == compare_labels).sum() 
    
    test_loss = sum_test_loss / float(len(dataloader))
    test_acc = float(correct) / float(total)
    
    return test_loss.item(),test_acc

# Data

In [5]:
######## DATA #####################################################
import torch.utils.data as data
import matplotlib.pyplot as plt
import numpy as np

from torch.utils.data import Dataset


class PairDataset(Dataset):

    def __init__(self, data, bool_labels, digit_labels = None):
        self.images = data
        self.bool_labels = bool_labels
        
        if digit_labels is not None:
            self.digit_labels = digit_labels

    def __len__(self):
        # override the class method. return the length of data
        return len(self.bool_labels)

    def __getitem__(self, idx):
        # override the class method. return the item at the index(idx)
        if self.digit_labels is not None:
            sample = {"images" : self.images[idx],
                      "bool_labels" : self.bool_labels[idx],
                      "digit_labels" : self.digit_labels[idx]}
        else:
            sample = {"images" : self.images[idx],
                      "bool_labels" : self.bool_labels[idx]}
            
        return sample
    
class SingleDataset(Dataset):

    def __init__(self, data, digit_labels):
        self.images = data
        self.digit_labels = digit_labels

    def __len__(self):
        # override the class method. return the length of data
        return len(self.digit_labels)

    def __getitem__(self, idx):
        # override the class method. return the item at the index(idx)
        sample = {"images" : self.images[idx],
                  "digit_labels" : self.digit_labels[idx]}
            
        return sample



pairs = generate_pair_sets(NB_SAMPLES)

train_dataset = PairDataset(pairs[0], pairs[1], pairs[2])
train_dataloader = data.DataLoader(dataset=train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True)

test_dataset = PairDataset(pairs[3], pairs[4], pairs[5])
test_dataloader = data.DataLoader(dataset=test_dataset, batch_size=TEST_BATCH_SIZE, shuffle=True)

gridsearch_params = [
    (kernel_size, nb_channels, fc, alpha)
   
    for kernel_size in [3,5]
    for nb_channels in [2,4,8,16,24,48]
    for fc in [32,64,128, 256]
    for alpha in np.linspace(0, 1, 5)
]

##############################################################

# Grid Search

In [6]:
from tempfile import TemporaryFile
outfile = TemporaryFile()
a=np.array([1,2])
np.savez("a",a)

In [8]:
cuda = torch.cuda.is_available()
if cuda:
    print("CUDA available")
else:
    print("NO CUDA")


lr = 0.001
epochs = 20
rounds = 5
round_results = [] #3D

for i in range(rounds):
    results = [] #training_losses, training_acc, test_losses, test_acc
    
    print('round {0} start'.format(i))
    
    for kernel_size, c, n, a in gridsearch_params:
        
        model = siamese_net(weight_sharing = True , architecture = 5, 
                            nb_channels = c, nb_hidden = n, k = kernel_size )
        training_losses, training_acc, test_losses, test_acc = train_siamese(model,
                                                                             train_dataloader, 
                                                                             test_dataloader, 
                                                                             epochs,
                                                                             lr, 
                                                                             alpha = a,
                                                                             aux_loss = True)
        
        print('With parameters {0} kernel size, {1} nb_channels, {2} nb_hidden fc,  {3} alpha: '.format(kernel_size, c, n, a))
        final_test_loss, final_test_loss_acc = test_siamese(model, test_dataloader, aux_loss = True)
        print("On the test set we obtain a loss of {:.2f} and an accuracy of {:.2f}".format(final_test_loss,final_test_loss_acc))
        
        results.append([training_losses, training_acc, test_losses, test_acc])
    
    
    print('round {0} end'.format(i))
    round_results.append(results)

NO CUDA
round 0 start
Start training with 20 epochs, a learning rate of 0.001 and BCELoss() as loss function
With auxilary loss function
With weight sharing


KeyboardInterrupt: 

In [ ]:
np.savez("results",round_results)